<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Feature-engineering" data-toc-modified-id="Feature-engineering-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Feature engineering</a></span><ul class="toc-item"><li><span><a href="#Imputation" data-toc-modified-id="Imputation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Imputation</a></span></li><li><span><a href="#Date-indicator" data-toc-modified-id="Date-indicator-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Date indicator</a></span></li></ul></li><li><span><a href="#Get-final-train-database" data-toc-modified-id="Get-final-train-database-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get final train database</a></span><ul class="toc-item"><li><span><a href="#Feature-selection" data-toc-modified-id="Feature-selection-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Feature selection</a></span><ul class="toc-item"><li><span><a href="#Only-for-the-first--(old)-dataset" data-toc-modified-id="Only-for-the-first--(old)-dataset-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Only for the first  (old) dataset</a></span><ul class="toc-item"><li><span><a href="#Shift-target" data-toc-modified-id="Shift-target-2.1.1.1"><span class="toc-item-num">2.1.1.1&nbsp;&nbsp;</span>Shift target</a></span></li></ul></li><li><span><a href="#For-the-new-dataset-(the-one-given-at-3am-:)-)" data-toc-modified-id="For-the-new-dataset-(the-one-given-at-3am-:)-)-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>For the new dataset (the one given at 3am :) )</a></span></li></ul></li><li><span><a href="#Lag-features" data-toc-modified-id="Lag-features-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Lag features</a></span></li></ul></li></ul></div>

In [2]:
import pandas as pd


import seaborn as sns
import numpy as np
import sklearn
import scipy.stats

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
#path= ''

In [3]:
data = pd.read_csv('train.csv')
# data_test=pd.read_csv('test.csv')

C:\Users\noureddine.oulid\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (13,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
data.head()

,date,store_id,parent_chain_id,store_country,country_iso_code,region_id,store_region,store_segment,store_activity_name,items_first_enabled_date,...,store_cancellation,item_price,meals_refunded,rating_count,sum_rating_overall,item_view,no_unique_consumers,is_enabled,Département,target
0,2019-06-02,4,NaN,France,FR,FR10,Ile-De-France,Bakery,Commercial,2019-06-03,...,0,3.99,0,0,0,0,0,False,75,0
1,2019-06-03,4,NaN,France,FR,FR10,Ile-De-France,Bakery,Commercial,2019-06-03,...,0,3.99,0,0,0,0,5,True,75,0
2,2019-06-04,4,NaN,France,FR,FR10,Ile-De-France,Bakery,Commercial,2019-06-03,...,0,3.99,0,1,5,0,4,True,75,0
3,2019-06-05,4,NaN,France,FR,FR10,Ile-De-France,Bakery,Commercial,2019-06-03,...,0,3.99,0,1,5,0,2,True,75,0
4,2019-06-06,4,NaN,France,FR,FR10,Ile-De-France,Bakery,Commercial,2019-06-03,...,0,3.99,0,0,0,0,2,True,75,0


In [104]:
data.columns

Index(['date', 'store_id', 'parent_chain_id', 'store_country',
       'country_iso_code', 'region_id', 'store_region', 'store_segment',
       'store_activity_name', 'items_first_enabled_date',
       'store_first_saving_date', 'store_last_saving_date', 'item_id',
       'item_name', 'before_price', 'currency_code', 'pickup_start',
       'pickup_end', 'total_supply', 'declared_supply', 'manual_added_supply',
       'manual_removed_supply', 'meals_saved', 'consumer_cancellation',
       'store_cancellation', 'item_price', 'meals_refunded', 'rating_count',
       'sum_rating_overall', 'item_view', 'no_unique_consumers', 'is_enabled',
       'Département', 'target'],
      dtype='object')

In [55]:
data.iloc[:, [13]].value_counts()

item_name                 
Panier classique 3,99 €       122005
Midi                           48186
Soir                           46722
Panier à 3,99 €                40311
Petit-déjeuner                 25162
                               ...  
frites fraiches sous vide          2
Panier Chocolats de Paques         1
Plantes Extérieur - Soir           1
panier midi                        1
Plantes Intérieur - Soir           1
Length: 1027, dtype: int64

In [56]:
data.iloc[:, [32]].value_counts()

Département
75             756448
59             286574
69             268787
13             195885
33             188756
                ...  
35                343
79                196
17                147
97                147
na                147
Length: 110, dtype: int64

In [57]:
data.isnull().sum()*100/len(data) #features with missing values ? 

date                         0.000000
store_id                     0.000000
parent_chain_id             37.688262
store_country                0.000000
country_iso_code             0.000000
region_id                    0.051756
store_region                 0.051756
store_segment                0.934408
store_activity_name          0.000000
items_first_enabled_date     1.415918
store_first_saving_date      4.738635
store_last_saving_date       4.738635
item_id                      0.000000
item_name                   84.361789
before_price                 5.062078
currency_code                0.000000
pickup_start                33.246484
pickup_end                  33.246484
total_supply                 0.000000
declared_supply              0.000000
manual_added_supply          0.000000
manual_removed_supply        0.000000
meals_saved                  0.000000
consumer_cancellation        0.000000
store_cancellation           0.000000
item_price                   0.000000
meals_refund

In [59]:
data[['sum_rating_overall', 'rating_count']].describe()

,sum_rating_overall,rating_count
count,5.431248e+06,5.431248e+06
mean,2.638443e+00,5.917343e-01
std,5.782910e+00,1.276928e+00
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
75%,5.000000e+00,1.000000e+00
max,8.600000e+02,1.990000e+02


In [61]:
data['before_price'].describe()

count    5.156314e+06
mean     1.315829e+01
std      5.692013e+00
min      3.990000e+00
25%      1.200000e+01
50%      1.200000e+01
75%      1.200000e+01
max      1.650000e+02
Name: before_price, dtype: float64

In [63]:
data[['date', 'store_id', 'before_price']]

,date,store_id,before_price
0,2019-06-02,4,12.0
1,2019-06-03,4,12.0
2,2019-06-04,4,12.0
3,2019-06-05,4,12.0
4,2019-06-06,4,12.0
...,...,...,...
5431243,2020-04-28,75409,12.0
5431244,2020-04-29,75409,12.0
5431245,2020-04-30,75409,12.0
5431246,2020-04-29,75480,12.0


In [8]:
data[['date', 'store_id', 'before_price', 'target']
     ].loc[data['before_price'].isnull() == True]

,date,store_id,before_price,target
2358,2019-07-14,203,NaN,1
2359,2019-07-15,203,NaN,1
2360,2019-07-16,203,NaN,1
2361,2019-07-17,203,NaN,1
2362,2019-07-18,203,NaN,1
...,...,...,...,...
4003844,2020-03-27,72192,NaN,1
4003845,2020-03-28,72192,NaN,1
4003846,2020-03-29,72192,NaN,1
4003847,2020-03-30,72192,NaN,1


In [7]:
data['target'].loc[(data['before_price'].isnull() == True)
                   ].value_counts()/len(data['before_price'].isnull() == True)

0    0.034620
1    0.018234
Name: target, dtype: float64

In [82]:
data['target'].value_counts()  # imbalanced ?

0.0    4477063
1.0     954185
Name: target, dtype: int64

In [6]:
data.loc[data['before_price'].isnull() == True, 'target'].value_counts() / len(data.loc[data['before_price'].isnull() == True])

0    0.655011
1    0.344989
Name: target, dtype: float64

In [101]:
# looks like we're gonna impute it with  12
data['before_price'].value_counts()/len(data)

12.0     0.592530
10.0     0.121412
15.0     0.097634
30.0     0.021660
18.0     0.019948
           ...   
59.9     0.000002
96.0     0.000002
54.0     0.000002
49.9     0.000002
165.0    0.000002
Name: before_price, Length: 134, dtype: float64

In [102]:
# 12 is also the median (might be the default value used by stores)
data['before_price'].describe()

count    5.156314e+06
mean     1.315829e+01
std      5.692013e+00
min      3.990000e+00
25%      1.200000e+01
50%      1.200000e+01
75%      1.200000e+01
max      1.650000e+02
Name: before_price, dtype: float64

In [ ]:
data.loc[data['meals_saved'] > data['total_supply'],
         ['store_id']].value_counts()
# hmm ok some stores have a strange use of the app

# Feature engineering

## Imputation

In [119]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer(strategy='median')

impute_median = np.array(data['before_price']).reshape(-1, 1)

before_price_was_imputed = data['before_price'].isnull()
before_price_imputed = pd.DataFrame(my_imputer.fit_transform(impute_median))

## Date indicator 

In [ ]:
date = pd.DataFrame()

date['date'] = pd.to_datetime(data['date'])

date['dayOfWeek'] = date['date'].dt.day_name()

date_day_string = date['dayOfWeek']

date[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']] = pd.get_dummies(
    date_day_string)[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]

date['weekend'] = date_day_string.isin(['Saturday', 'Sunday']).replace({
    True: 1, False: 0})  # indicate weekend

In [163]:
date

,date,dayOfWeek,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,weekend
0,2019-06-02,Sunday,0,0,0,0,0,0,1,1
1,2019-06-03,Monday,1,0,0,0,0,0,0,0
2,2019-06-04,Tuesday,0,1,0,0,0,0,0,0
3,2019-06-05,Wednesday,0,0,1,0,0,0,0,0
4,2019-06-06,Thursday,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
5431243,2020-04-28,Tuesday,0,1,0,0,0,0,0,0
5431244,2020-04-29,Wednesday,0,0,1,0,0,0,0,0
5431245,2020-04-30,Thursday,0,0,0,1,0,0,0,0
5431246,2020-04-29,Wednesday,0,0,1,0,0,0,0,0


In [156]:
date_day_string

0             Sunday
1             Monday
2            Tuesday
3          Wednesday
4           Thursday
             ...    
5431243      Tuesday
5431244    Wednesday
5431245     Thursday
5431246    Wednesday
5431247     Thursday
Name: dayOfWeek, Length: 5431248, dtype: object

In [166]:
data.columns

Index(['date', 'store_id', 'parent_chain_id', 'store_country',
       'country_iso_code', 'region_id', 'store_region', 'store_segment',
       'store_activity_name', 'items_first_enabled_date',
       'store_first_saving_date', 'store_last_saving_date', 'item_id',
       'item_name', 'before_price', 'currency_code', 'pickup_start',
       'pickup_end', 'total_supply', 'declared_supply', 'manual_added_supply',
       'manual_removed_supply', 'meals_saved', 'consumer_cancellation',
       'store_cancellation', 'item_price', 'meals_refunded', 'rating_count',
       'sum_rating_overall', 'item_view', 'no_unique_consumers', 'is_enabled',
       'Département', 'target'],
      dtype='object')

In [180]:
data.groupby(['store_id', 'date']).count()

parent_chain_id  store_country  country_iso_code  \
store_id date                                                           
4        2019-06-02                0              1                 1   
         2019-06-03                0              1                 1   
         2019-06-04                0              1                 1   
         2019-06-05                0              1                 1   
         2019-06-06                0              1                 1   
...                              ...            ...               ...   
75409    2020-04-28                1              1                 1   
         2020-04-29                1              1                 1   
         2020-04-30                1              1                 1   
75480    2020-04-29                0              1                 1   
         2020-04-30                0              1                 1   

                     region_id  store_region  store_segment  \
store_id date                                                 
4        2019-06-02          1             1              1   
         2019-06-03          1             1              1   
         2019-06-04          1             1              1   
         2019-06-05          1             1              1   
         2019-06-06          1             1              1   
...                        ...           ...            ...   
75409    2020-04-28          1             1              1   
         2020-04-29          1             1              1   
         2020-04-30          1             1              1   
75480    2020-04-29          1             1              1   
         2020-04-30          1             1              1   

                     store_activity_name  items_first_enabled_date  \
store_id date                                                        
4        2019-06-02                    1                         1   
         2019-06-03                    1                         1   
         2019-06-04                    1                         1   
         2019-06-05                    1                         1   
         2019-06-06                    1                         1   
...                                  ...                       ...   
75409    2020-04-28                    1                         1   
         2020-04-29                    1                         1   
         2020-04-30                    1                         1   
75480    2020-04-29                    1                         1   
         2020-04-30                    1                         1   

                     store_first_saving_date  store_last_saving_date  ...  \
store_id date                                                         ...   
4        2019-06-02                        1                       1  ...   
         2019-06-03                        1                       1  ...   
         2019-06-04                        1                       1  ...   
         2019-06-05                        1                       1  ...   
         2019-06-06                        1                       1  ...   
...                                      ...                     ...  ...   
75409    2020-04-28                        1                       1  ...   
         2020-04-29                        1                       1  ...   
         2020-04-30                        1                       1  ...   
75480    2020-04-29                        1                       1  ...   
         2020-04-30                        1                       1  ...   

                     store_cancellation  item_price  meals_refunded  \
store_id date                                                         
4        2019-06-02                   1           1               1   
         2019-06-03                   1           1               1   
         2019-06-04                   1           1   

In [200]:
data.groupby(['date', 'store_id']).count()['item_id'].value_counts()*100/len(data)
# ok we have a problem  : stores have many items, we'll use groupby

1     91.157741
2      3.581037
3      0.354081
4      0.105906
5      0.030177
6      0.006242
8      0.000313
7      0.000313
10     0.000092
11     0.000018
9      0.000018
Name: item_id, dtype: float64

In [224]:
data['store_activity_name'].value_counts() #useles, only 3 categories ? might don't discriminate that much 

Commercial    4136349
Unknown       1157206
Temporary      137693
Name: store_activity_name, dtype: int64

In [225]:
data['currency_code'].value_counts() # only 1 value, drop it

EUR    5431248
Name: currency_code, dtype: int64

In [226]:
# interesting to explore it with basic descriptive tools
data['meals_refunded'].value_counts()

0     5370867
1       41272
2       11945
3        3825
4        1657
5         916
6         290
7         166
8         110
10         73
9          64
12         13
11          9
13          5
14          5
17          5
20          5
15          4
16          4
18          4
22          3
33          2
19          1
24          1
31          1
35          1
Name: meals_refunded, dtype: int64

# Get final train database

## Feature selection

In [3]:
# keep features
X = data[['date', 'store_id', 'parent_chain_id', 'store_region', 'store_segment',
          'store_first_saving_date', 'before_price', 'total_supply', 'declared_supply', 'manual_added_supply',
          'manual_removed_supply', 'meals_saved', 'consumer_cancellation',
          'store_cancellation', 'item_price', 'meals_refunded', 'rating_count',
          'sum_rating_overall', 'item_view', 'no_unique_consumers',
          'Département', 'target']]

In [4]:
# dropna and junked data
X = X.dropna(subset=['store_first_saving_date'])
X['parent_chain_id'] = X['parent_chain_id'].fillna(0)
X = X[X['meals_saved'] <= X['total_supply']]
print('1', len(X))
# create diff saved and diff first
X['diff_saved'] = ((X['total_supply'] - X['meals_saved']) /
                   X['total_supply']).fillna(0)
X['diff_first'] = (pd.to_datetime(X['date']) -
                   pd.to_datetime(X['store_first_saving_date'])).dt.days
X.drop(columns=['store_first_saving_date'], inplace=True)

1 3801885


In [5]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer(strategy='median')

impute_median = np.array(X['before_price']).reshape(-1, 1)

before_price_was_imputed = X['before_price'].isnull()
before_price_imputed = pd.DataFrame(my_imputer.fit_transform(impute_median))

print(len(X.columns))
X.drop(columns=['before_price'], axis=1, inplace=True)
print('2', len(X))
print(len(X.columns))

X['before_price'] = np.array(before_price_imputed)
print('3', len(X))

date = pd.DataFrame()
date['date'] = pd.to_datetime(X['date'])
date['dayOfWeek'] = date['date'].dt.day_name()
date_day_string = date['dayOfWeek']
date[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']] = pd.get_dummies(
    date_day_string)[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]
date['weekend'] = date_day_string.isin(
    ['Saturday', 'Sunday']).replace({True: 1, False: 0})

X.drop(columns=['date'], inplace=True)
print(len(X))

23
2 3801885
22
3 3801885
3801885


In [6]:
# create dummies for region, department
region_dummies = pd.get_dummies(X['store_region'])
segment_dummies = pd.get_dummies(X['store_segment'])
X = pd.concat([date, X, region_dummies, segment_dummies], axis=1)
X.drop(columns=['dayOfWeek', 'store_region', 'store_segment'], inplace=True)
X['Département'] = X['Département'].astype('int64')

In [35]:
X.columns

Index(['date', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
       'Saturday', 'Sunday', 'weekend', 'date', 'store_id', 'parent_chain_id',
       'before_price', 'total_supply', 'declared_supply',
       'manual_added_supply', 'manual_removed_supply', 'meals_saved',
       'consumer_cancellation', 'store_cancellation', 'item_price',
       'meals_refunded', 'rating_count', 'sum_rating_overall', 'item_view',
       'no_unique_consumers', 'Département', 'target', 'diff_saved',
       'diff_first', 'Alsace', 'Aquitaine', 'Auvergne', 'Basse-Normandie',
       'Bourgogne', 'Bretagne', 'Centre - Val De Loire', 'Champagne-Ardenne',
       'Corse', 'Franche-Comté', 'Haute-Normandie', 'Ile-De-France',
       'Languedoc-Roussillon', 'Limousin', 'Lorraine', 'Midi-Pyrénées',
       'Nord-Pas De Calais', 'Pays De La Loire', 'Picardie',
       'Poitou-Charentes', 'Provence-Alpes-Côte D’Azur', 'Rhône-Alpes',
       'Bakery', 'Beverages', 'Buffet', 'Butcher', 'Cafeteria', 'Canteen',
       

### Only for the first  (old) dataset 

In [9]:
# this was useful before we were provided with a new database, cleared of double items etc..
# don't run it for the new  train dataset

X_sum = X[['date', 'store_id', 'total_supply', 'declared_supply',
           'manual_added_supply', 'manual_removed_supply', 'meals_saved',
           'consumer_cancellation', 'store_cancellation', 'meals_refunded', 'rating_count', 'sum_rating_overall', 'item_view',
           'no_unique_consumers']]

In [10]:
# Same here
X_mean = X[['date', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
            'Friday', 'Saturday', 'Sunday', 'weekend', 'store_id',
            'parent_chain_id', 'item_price',
            'target', 'diff_saved',
            'diff_first', 'before_price', 'Alsace',
            'Aquitaine', 'Auvergne', 'Basse-Normandie', 'Bourgogne', 'Bretagne',
            'Centre - Val De Loire', 'Champagne-Ardenne', 'Corse', 'Franche-Comté',
            'Haute-Normandie', 'Ile-De-France', 'Languedoc-Roussillon', 'Limousin',
            'Lorraine', 'Midi-Pyrénées', 'Nord-Pas De Calais', 'Pays De La Loire',
            'Picardie', 'Poitou-Charentes', 'Provence-Alpes-Côte D’Azur',
            'Rhône-Alpes', 'Bakery', 'Beverages', 'Buffet', 'Butcher', 'Cafeteria',
            'Canteen', 'Caterer', 'Fast Food / Take away restaurant', 'Fish shop',
            'Florist', 'Fruits & vegetables store', 'Gas station', 'Hotel',
            'Manufacturer', 'Market stall', 'Other', 'Other Specialists', 'Pastry',
            'Supermarket Big', 'Supermarket Medium', 'Supermarket Small', 'Sushi',
            'Traditional Restaurant']]

In [ ]:
X_sum=X_sum.groupby(['store_id', 'date'], as_index=False).sum()

In [12]:
X_mean=X_mean.groupby(['store_id', 'date'], as_index=False).mean()

In [ ]:
X_mean.drop(columns=['store_id', 'date'], inplace=True)
X_final = pd.concat([X_sum, X_mean], axis=1)

#### Shift target 

In [16]:
# target is actualy wrong (don't take multiple items and is too late )
X_temp = X_final[['store_id', 'target']]
X_ = np.zeros((len(X_temp), len(X_temp.columns)))
compteur = 0
for i in tqdm(list(X_temp.store_id.unique())):  # make sure we shift in for a given store
    df = X_temp[X_temp['store_id'] == i].copy()
    df['target'] = df['target'].shift(-30)
    buf = df.to_numpy()  # using numpy to speed up calculation
    X_[compteur:compteur+len(buf), :] = buf
    compteur += len(buf)
    #X_ = pd.concat([df, X_])

In [17]:
target_shifted=pd.DataFrame(X_, columns=['store_id', 'target_shifted'])['target_shifted']

In [18]:
X_train = pd.concat([X_final, target_shifted], axis=1)
X_train['target_shifted'] = X_train['target_shifted'].fillna(0)

In [22]:
X_train.to_csv('X_train.csv', index=False)

### For the new dataset (the one given at 3am :) ) 

In [7]:
X_final=X

In [8]:
len(X_final)

3801885

## Lag features

In [40]:
features_to_lag = ['store_id', 'total_supply', 'meals_saved', 'item_view']

In [41]:
X_final[features_to_lag]

,store_id,total_supply,meals_saved,item_view
0,4,2,0,0
6,4,0,0,0
13,4,0,0,0
16,4,4,4,0
17,4,4,4,0
...,...,...,...,...
4003927,73292,20,20,185
4003928,73372,0,0,0
4003929,73372,2,2,29
4003930,73398,7,6,160


In [42]:
X_temp = X_final[features_to_lag]
X_ = np.zeros((len(X_temp), 49))
compteur = 0
for i in tqdm(list(X_temp.store_id.unique())):
    df = X_temp[X_temp['store_id'] == i].copy()
    for i in range(1, 16):
        for col in ['total_supply', 'meals_saved', 'item_view']:
            df["{}_lag_{}".format(col, i)] = df[col].shift(i)
    buf = df.to_numpy()
    X_[compteur:compteur+len(buf), :] = buf
    compteur += len(buf)

In [43]:
X_shifted = pd.DataFrame(X_, columns=df.columns)

In [44]:
X_shifted.to_csv('X_shifted.csv', index=False)

In [9]:
X_shifted=pd.read_csv('X_shifted.csv') #notebook  crashed 

In [20]:
X_final.drop(columns=['store_id'], axis=1, inplace=True)
X_final.reset_index(drop=True, inplace=True)

In [21]:
X_final.head()

,date,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,weekend,parent_chain_id,...,Manufacturer,Market stall,Other,Other Specialists,Pastry,Supermarket Big,Supermarket Medium,Supermarket Small,Sushi,Traditional Restaurant
0,2019-06-02,0,0,0,0,0,0,1,1,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2019-06-08,0,0,0,0,0,1,0,1,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2019-06-15,0,0,0,0,0,1,0,1,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2019-06-18,0,1,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2019-06-19,0,0,1,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
X_shifted.head()

,store_id,total_supply,meals_saved,item_view,total_supply_lag_1,meals_saved_lag_1,item_view_lag_1,total_supply_lag_2,meals_saved_lag_2,item_view_lag_2,...,item_view_lag_12,total_supply_lag_13,meals_saved_lag_13,item_view_lag_13,total_supply_lag_14,meals_saved_lag_14,item_view_lag_14,total_supply_lag_15,meals_saved_lag_15,item_view_lag_15
0,4.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,0.0,0.0,0.0,2.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4.0,4.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
X_train=pd.concat([X_final,X_shifted],axis=1)

In [ ]:
# adding a feature capturing ratings evolution w.r.t time

sum_rate, nb_rates = 0, 0
mem = 0
res_moy = []
for i, row in X_train.iterrows():
    if i % 100000 == 0:
        print(i)
    if row['store_id'] != mem:
        sum_rate, nb_rates = 0, 0
        mem = row['store_id']
    sum_rate += row['sum_rating_overall']
    nb_rates += row['rating_count']
    if nb_rates > 0:
        res_moy.append(sum_rate/nb_rates)
    else:
        res_moy.append(4.45)

In [11]:
#X_train['churn_duration']=X_train['target_shifted'].cumsum()*X_train['target_shifted']
#X_train['rate_evolution']=res_moy 
a = X_train['target'] != 0
X_train['churn_duration'] = a.cumsum()-a.cumsum().where(~a).ffill().fillna(0).astype(int) 
#how many consecutive days do store churn?

In [ ]:
X_train[X_train['store_id']==22].loc[400:450, ['target', 'churn_duration']] #check churn duration is actually right

In [12]:
X_train[['store_id', 'rate_evolution', 'churn_duration']].to_csv(
    'churn_for_clem.csv', index=False)  # for descr stat

In [17]:
X_train.columns #make sure everything is fine :) 

Index(['store_id', 'date', 'total_supply', 'declared_supply',
       'manual_added_supply', 'manual_removed_supply', 'meals_saved',
       'consumer_cancellation', 'store_cancellation', 'meals_refunded',
       ...
       'meals_saved_lag_13', 'item_view_lag_13', 'total_supply_lag_14',
       'meals_saved_lag_14', 'item_view_lag_14', 'total_supply_lag_15',
       'meals_saved_lag_15', 'item_view_lag_15', 'rate_evolution',
       'churn_duration'],
      dtype='object', length=125)

In [24]:
X_train.to_csv('X_train_final_duration.csv', index=False)